In [ ]:
import pandas as pd
from pandas.core.dtypes.common import is_datetime64_any_dtype

vessel_positions = pd.read_csv('data/vessel_positions.csv')
vessel_characteristics = pd.read_csv('data/vessel_characteristics.csv')

In [ ]:
'''
Convert a column that mixes Excel-serial numbers and “DD/MM/YYYY HH:MM” strings into one UTC-timestamp Series.
Rounds timings to the nearest second.

Returns unchanged if the column is already a datatime dtype. Otherwise, returns a Series of dtype datetime64[ns, UTC]; unparseable rows become NaT.
'''

def to_timestamp(dataframe, column_name):

    s = dataframe[column_name]

    if is_datetime64_any_dtype(s):
        return s

    numbers = pd.to_numeric(s, errors="coerce") # floats where possible, NaN otherwise
    dt1 = pd.to_datetime(numbers, unit="D", origin="1899-12-30", utc=True) # convert floats to timestamps

    dates = s[numbers.isna()] # take the non-numeric rows

    # convert dates to timestamps
    dt2 = pd.to_datetime(dates, format="%d/%m/%Y %H:%M", dayfirst=True, utc=True, errors="coerce")

    final_column = dt1.fillna(dt2) # merge dt1 and dt2 into a single series of timestamps

    # round the timings to the nearest second
    final_column = final_column.dt.round('s')

    return final_column

The to_timestamp function is used convert datetime columns of string objects to datetime64[ns, UTC] dtype. This data type conversion provides accurate comparisons, date arithmetic, timezone handling, faster operations, less storage, and cleaner/built-in support (pandas and other libraries).

Four additional functions, object_to_string, float_to_Int64, std_to_boolean, and to_date, were implemented to convert columns from the default data types to the data types that suit the contents of each column best.

The strip_html function is used to clean axs_notes_dry column from <br> html elements.

All columns which were mistakenly altered to have the boolean data type because they were empty were adjusted back to the most suitable data type.

In [ ]:
'''
Takes a dataframe, searches for text columns of object dtype and converts them to the string dtype.
Returns the modified dataframe.
'''

def object_to_string(dataframe):

    obj_cols = dataframe.select_dtypes(include=["object"]).columns

    for column in obj_cols:
        if all(isinstance(v, str) for v in dataframe[column].dropna()):
            dataframe[column] = dataframe[column].astype("string")

    return dataframe

In [ ]:
'''
Takes a dataframe, searches for integer-like columns of float dtype and converts them to Pandas' Int64 dtype.
Returns the modified dataframe.
'''

import numpy as np

def float_to_Int64(dataframe):

    float_cols = dataframe.select_dtypes(include=["float"]).columns

    for column in float_cols:
        if np.isclose(dataframe[column].dropna() % 1, 0).all():
            dataframe[column] = dataframe[column].astype("Int64")

    return dataframe

In [ ]:
'''
Takes a dataframe, searches for columns of the integer data type that are boolean in nature and converts them to boolean dtype.
Standardizes boolean columns from 'Y'/'N' to True/False
Returns the modified dataframe.
'''

def std_to_boolean(dataframe):
    int_cols = dataframe.select_dtypes(include=["integer"]).columns

    for column in int_cols:
        if dataframe[column].dropna().isin([0, 1]).all():
            dataframe[column] = dataframe[column].astype("boolean")

    str_cols = dataframe.select_dtypes(include=["string"]).columns

    for column in str_cols:
        # Standardise to upper-case and strip whitespace once
        col_std = dataframe[column].dropna().str.strip().str.upper()

        if col_std.isin(["Y", "N"]).all():
            dataframe[column] = (
                dataframe[column].str.strip()
                   .str.upper()
                   .map({"Y": True, "N": False})
                   .astype("boolean")
            )

    return dataframe

In [ ]:
'''
Takes a dataframe column of dates, converts it from object dtype to datetime dtype.
 Returns the modified column.
'''

def to_date(column):

    patterns = ["%d/%m/%Y", "%Y-%m-%d"]

    parsed = pd.Series(pd.NaT, index=column.index) # start all NaT

    for fmt in patterns:
        mask = parsed.isna() # rows still unparsed
        parsed.loc[mask] = pd.to_datetime(
            column.loc[mask],
            format=fmt,
            dayfirst=(fmt == "%d/%m/%Y"),
            errors="coerce"
        )

    return parsed

In [ ]:
'''
Takes a dataframe column.
Removes <br>, <br/>, <br /> (case-insensitive) and collapse multiple spaces.
Returns the modified column.
'''

import pandas as pd
import re

def strip_html(column):
    return (
           column.str.replace(r"<br\s*/?>", " ", regex=True, flags=re.I)
           .str.replace(r"\s+", " ", regex=True)
           .str.strip()
    )

In [ ]:
# vessel_positions data cleaning

# Convert datetime columns of string objects to datetime64[ns, UTC] dtype

datetime_cols = ['zone_entry_time', 'zone_out_time', 'port_entry_time',
        'port_out_time', 'operation_location_entry_time', 'operation_location_out_time',
        'waiting_zone_entry_time', 'waiting_zone_out_time', 'ais_eta', 'last_seen']

for col in datetime_cols:
    vessel_positions[col] = to_timestamp(vessel_positions, col)

date_cols = ['status_date_time', 'updated_at']

for col in date_cols:
    vessel_positions[col] = pd.to_datetime(vessel_positions[col], format="%d/%m/%Y", dayfirst=True, errors="coerce")

# Convert boolean in nature columns from integer dtype to boolean dtype
std_to_boolean(vessel_positions)

# Convert text columns from object dtype to string dtype
object_to_string(vessel_positions)

# Convert integer-like float columns to Int64
float_to_Int64(vessel_positions)

In [ ]:
# vessel_characteristics data cleaning

# Convert datetime columns of string objects to datetime64[ns, UTC] dtype

datetime_cols = ['update_date']

for col in datetime_cols:
    vessel_characteristics[col] = to_timestamp(vessel_characteristics, col)

date_cols = ['built', 'demolition_date', 'keel_laying_date', 'launching_date', 'reported_date', 'cancelled_date', 'last_dd_date', 'next_dd_date', 'last_ss_date', 'next_ss_date', 'conversion_date']

for col in date_cols:
    vessel_characteristics[col] = to_date(vessel_characteristics[col])

# Convert text columns from object dtype to string dtype
object_to_string(vessel_characteristics)

# Convert integer-like float columns to Int64
float_to_Int64(vessel_characteristics)

# Convert boolean in nature columns from integer dtype to boolean dtype
# Separate open_hatches into categorical and boolean columns first

type_map = {
    "BOX":  "BOX",
    "OHBS": "OHBS",
    "Y":    "generic",
    "N":    "none"
}
vessel_characteristics["open_hatches"] = (
    vessel_characteristics["open_hatches"].map(type_map)
       .astype("category")
)

bool_map = {"BOX": True, "OHBS": True, "generic": True, "none": False}
bool_series = (
    vessel_characteristics["open_hatches"]
      .map(bool_map)
      .astype("boolean")
)

pos = vessel_characteristics.columns.get_loc("open_hatches")

vessel_characteristics.insert(
    loc=pos + 1,
    column="open_hatches_bool",
    value=bool_series
)

std_to_boolean(vessel_characteristics)

# return empty column to original dtype (or delete them)

vessel_characteristics["technical_manager"].astype("string")
vessel_characteristics["parent_shipowner"].astype("string")
vessel_characteristics["doc_holder"].astype("string")
vessel_characteristics["crew_manager"].astype("string")
vessel_characteristics["ism_manager"].astype("string")
vessel_characteristics["technical_manager_id"].astype("Int64")
vessel_characteristics["technical_manager_flag"].astype("string")
vessel_characteristics["technical_manager_flag_id"].astype("Int64")
vessel_characteristics["parent_shipowner_id"].astype("Int64")
vessel_characteristics["parent_shipowner_flag"].astype("string")
vessel_characteristics["parent_shipowner_flag_id"].astype("Int64")
vessel_characteristics["doc_holder_id"].astype("Int64")
vessel_characteristics["doc_holder_flag"].astype("string")
vessel_characteristics["doc_holder_flag_id"].astype("Int64")
vessel_characteristics["crew_manager_id"].astype("Int64")
vessel_characteristics["crew_manager_flag"].astype("string")
vessel_characteristics["crew_manager_flag_id"].astype("Int64")
vessel_characteristics["ism_manager_id"].astype("Int64")
vessel_characteristics["ism_manager_flag"].astype("string")
vessel_characteristics["ism_manager_flag_id"].astype("Int64")
vessel_characteristics["cranes_position"].astype("string")
vessel_characteristics["private_comments"].astype("string")
vessel_characteristics["private_tags"].astype("string")
vessel_characteristics["private_notes"].astype("string")

# strip the html tags from axs_notes_dry

vessel_characteristics["axs_notes_dry"] = strip_html(vessel_characteristics["axs_notes_dry"])